In [1]:
import os
import gmsh
gmsh.initialize()               # Need to initialize gmsh
import pygmsh
import meshio
import numpy as np

import matplotlib.pyplot as plt
from scipy import interpolate


import os
import meshio

import ufl
import dolfinx
from mpi4py import MPI
from petsc4py import PETSc # https://www.mcs.anl.gov/petsc/petsc4py-current/docs/apiref/index.html
from slepc4py import SLEPc # https://slepc.upv.es/slepc4py-current/docs/apiref/index.html

import time
import numpy as np
import matplotlib.pyplot as plt

## Two Rectangles

In [2]:
gmsh.model.add("TwoRect")

bottom_base_tag = gmsh.model.occ.addRectangle(0, 0, 0, 2, 5)
gmsh.model.occ.extrude([(2,bottom_base_tag)], 0, 0, 0.2)

top_base_tag = gmsh.model.occ.addRectangle(0, 5, 0, 2, 5)
gmsh.model.occ.extrude([(2,top_base_tag)], 0, 0, 0.2)

gmsh.model.occ.fragment([(3,1),(3,2)],[])
gmsh.model.occ.synchronize()

rect_3Dentities = gmsh.model.getEntities(dim=3)

print(rect_3Dentities)

# gmsh.model.geo.removeAllDuplicates()
# gmsh.model.mesh.removeDuplicateNodes()

rect_left_marker = gmsh.model.addPhysicalGroup(3, [rect_3Dentities[0][1]])
gmsh.model.setPhysicalName(3, rect_left_marker, "Left Volume")

rect_right_marker = gmsh.model.addPhysicalGroup(3, [rect_3Dentities[1][1]])
gmsh.model.setPhysicalName(3, rect_right_marker, "Right Volume")


rect_2Dentities = gmsh.model.getEntities(dim=2)
rect_marker = gmsh.model.addPhysicalGroup(2, np.array(rect_2Dentities)[:,1])
gmsh.model.setPhysicalName(2, rect_marker, "Side")

gmsh.option.setNumber('Mesh.MeshSizeMax', 0.1)

gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)

#os.path.join("MeshGeometry", "TwoRect.msh")
gmsh.write(os.path.join("MeshGeometry", "TwoRect.msh"))


[(3, 1), (3, 2)]
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 30%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 40%] Meshing curve 8 (Line)
Info    : [ 40%] Meshing curve 9 (Line)
Info    : [ 50%] Meshing curve 10 (Line)
Info    : [ 50%] Meshing curve 11 (Line)
Info    : [ 60%] Meshing curve 12 (Line)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 70%] Meshing curve 15 (Line)
Info    : [ 80%] Meshing curve 16 (Line)
Info    : [ 80%] Meshing curve 17 (Line)
Info    : [ 90%] Meshing curve 18 (Line)
Info    : [ 90%] Meshing curve 19 (Line)
Info    : [100%] Meshing curve 20 (Line)
Info    : Done meshing 1D (Wall 0.0023595s, CPU 0.003484s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay)
Info

In [3]:
from gmsh_helpers import gmsh_model_to_mesh
tworect_mesh, tworect_cell_tags, tworect_facet_tags = gmsh_model_to_mesh(gmsh.model, cell_data=True, facet_data=True, gdim=3)


In [5]:
def epsilon(u):
    # Strain
    return 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)

def strain2voigt(e):
    return ufl.as_vector([e[0,0],e[1,1],e[2,2],e[1,2],e[0,2],e[0,1]])

def voigt2stress(s):
    return ufl.as_tensor([[s[0], s[5], s[4]],
                         [s[5], s[1], s[3]],
                         [s[4], s[3], s[2]]])

# This function allow flexibility (marginal) by taking in another C_scale value
def sigma(u, C, C_scale):
    C = ufl.as_matrix(C)  
    return voigt2stress(C_scale*ufl.dot(C,strain2voigt(epsilon(u))))

In [6]:
def modalAnalysisOrthotropic(geom_mesh, geom_cell_tags, C, params):
    
    rho = params['rho']
    elementOrder = params['elementOrder']
    N_eig = params['N_eig']
        

    # Define C_scale which is a scalar value that will change depending on cell selection
    Q = dolfinx.fem.FunctionSpace(geom_mesh, ("DG", 0))
    C_scale = dolfinx.fem.Function(Q)
    rect_left_cells = geom_cell_tags.indices[geom_cell_tags.values==rect_left_marker]
    rect_right_cells = geom_cell_tags.indices[geom_cell_tags.values==rect_right_marker]
    
    # HERE we explicity set 1 and 0.1 as different material parameters for different parts of the mesh
    C_scale.x.array[rect_left_cells] = np.full(len(rect_left_cells), 1)
    C_scale.x.array[rect_right_cells] = np.full(len(rect_right_cells), 0.1)
    
    
    # Define Weak Formulation
    time_start = time.process_time()

    V = dolfinx.fem.VectorFunctionSpace(geom_mesh, ("Lagrange", elementOrder))
    u_tr = ufl.TrialFunction(V) 
    u_test = ufl.TestFunction(V)

    a = ufl.inner(sigma(u_tr, C, C_scale), epsilon(u_test))*ufl.dx
    m = rho*ufl.inner(u_tr, u_test)*ufl.dx
    
    a, m = dolfinx.fem.form(a), dolfinx.fem.form(m)

    A = dolfinx.fem.petsc.assemble_matrix(a)
    A.assemble()

    M = dolfinx.fem.petsc.assemble_matrix(m)
    M.assemble()

    
    eigensolver = SLEPc.EPS().create()
    eigensolver.setOperators(A,M)
    
    # Non-hermite problem
    eigensolver.setProblemType(SLEPc.EPS.ProblemType.GHEP)
    #eigensolver.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_MAGNITUDE)
    eigensolver.setTolerances(tol=1.e-11, max_it=1000)
    eigensolver.setDimensions(N_eig)
    
    eigensolver.setFromOptions()
    st = eigensolver.getST()
    st.setType(SLEPc.ST.Type.SINVERT)
    
    eigensolver.solve()
    
    nconv = eigensolver.getConverged()
    print( "Number of converged eigenpairs %d" % nconv )
    
    time_elapsed = (time.process_time() - time_start)
    print("Time for modal analysis: {} ".format(time_elapsed))
    
    with dolfinx.io.XDMFFile(MPI.COMM_WORLD, os.path.join("OutputModalShapes", "tworect.xdmf"), "w") as my_file_xdmf:
        my_file_xdmf.write_mesh(geom_mesh)
        
        vr, vi = A.createVecs()
        
        # Get Eigenmodes
        for i in range(0, nconv):
            # Get i-th eigenvalue and eigenvector
            eig_val = eigensolver.getEigenpair(i, vr, vi)
            print("{} : {}".format(i, eig_val))
            
            if (~np.isclose(eig_val.real, 1.0)):
                #  Calculation of eigenfrequency from real part of eigenvalue
                freq_3D = np.sqrt(eig_val.real)/2/np.pi
                print("Eigenfrequency: {0:8.5f} [Hz]".format(freq_3D))

                u = dolfinx.fem.Function(V, name="Eigenvector " + str(i))
                u.vector.setArray(vr.array)
                my_file_xdmf.write_function(u)

In [7]:
def getC(params):
    # Material Parameters
    E_x, E_y, E_z  = params['E_x'], params['E_y'], params['E_z']
    nu_xy, nu_xz, nu_yx, nu_yz, nu_zx, nu_zy = params['nu_xy'], params['nu_xz'], params['nu_yx'], params['nu_yz'], params['nu_zx'], params['nu_zy'], 
    G_xy, G_yz, G_zx = params['G_xy'], params['G_yz'], params['G_zx']

    # Make orthotropic stiffness matrix
    delta = (1- nu_xy*nu_yx - nu_yz*nu_zy - nu_zx*nu_xz - 2*nu_xy*nu_yz*nu_zx)/(E_x * E_y * E_z)

    C = np.zeros((6,6))
    C[0,0] = (1-nu_yz*nu_zy)/(E_y*E_z*delta)
    C[0,1] = (nu_yx+nu_zx*nu_yz)/(E_y*E_z*delta)
    C[0,2] = (nu_zx+nu_yx*nu_zy)/(E_y*E_z*delta)
    C[1,0] = (nu_xy+nu_xz*nu_zy)/(E_z*E_x*delta)
    C[1,1] = (1-nu_zx*nu_xz)/(E_z*E_x*delta)
    C[1,2] = (nu_zy+nu_zx*nu_xy)/(E_z*E_x*delta)
    C[2,0] = (nu_xz+nu_xy*nu_yz)/(E_x*E_y*delta)
    C[2,1] = (nu_yz+nu_xz*nu_yx)/(E_x*E_y*delta)
    C[2,2] = (1-nu_xy*nu_yx)/(E_x*E_y*delta)
    C[3,3] = 2*G_yz
    C[4,4] = 2*G_zx
    C[5,5] = 2*G_xy
    
    return C

In [8]:
EnglemanSpruce = {}

EnglemanSpruce['E_x'] = 8.9E9
EnglemanSpruce['E_y'] = 8.9E9
EnglemanSpruce['E_z'] = 8.9E9
EnglemanSpruce['nu_xy'] = .4
EnglemanSpruce['nu_xz'] = .4
EnglemanSpruce['nu_yx'] = .4
EnglemanSpruce['nu_yz'] = .4
EnglemanSpruce['nu_zx'] = .4
EnglemanSpruce['nu_zy'] = .4
EnglemanSpruce['G_yz'] = 0.1*EnglemanSpruce['E_x']
EnglemanSpruce['G_zx'] = 0.1*EnglemanSpruce['E_x']
EnglemanSpruce['G_xy'] = 0.1*EnglemanSpruce['E_x']

C = getC(EnglemanSpruce)

In [9]:
FEMparams = {}
FEMparams['rho'] = 435.1101985
FEMparams['elementOrder'] = 2
FEMparams['N_eig'] = 24

In [11]:
modalAnalysisOrthotropic(tworect_mesh, tworect_cell_tags, C, FEMparams)

Number of converged eigenpairs 30
Time for modal analysis: 66.043478835 
0 : (8.461640397534364e-08+0j)
Eigenfrequency:  0.00005 [Hz]
1 : (1.5065804653108683e-07+0j)
Eigenfrequency:  0.00006 [Hz]
2 : (4.0694924774097224e-07+0j)
Eigenfrequency:  0.00010 [Hz]
3 : (4.660576532826819e-07+0j)
Eigenfrequency:  0.00011 [Hz]
4 : (8.622646091065661e-07+0j)
Eigenfrequency:  0.00015 [Hz]
5 : (1.1935302101405655e-06+0j)
Eigenfrequency:  0.00017 [Hz]
6 : (591.9416457817478+0j)
Eigenfrequency:  3.87222 [Hz]
7 : (1565.1356123172015+0j)
Eigenfrequency:  6.29646 [Hz]
8 : (7111.4358254144245+0j)
Eigenfrequency: 13.42143 [Hz]
9 : (10894.822958611405+0j)
Eigenfrequency: 16.61232 [Hz]
10 : (27134.668129704543+0j)
Eigenfrequency: 26.21696 [Hz]
11 : (28212.53664617457+0j)
Eigenfrequency: 26.73260 [Hz]
12 : (36584.912134930084+0j)
Eigenfrequency: 30.44186 [Hz]
13 : (62209.28894486115+0j)
Eigenfrequency: 39.69609 [Hz]
14 : (63719.67726725926+0j)
Eigenfrequency: 40.17510 [Hz]
15 : (127357.82725732085+0j)
Eigenf